In [2]:
import os
import random
import cv2
import numpy as np
from pdf2image import convert_from_path,pdfinfo_from_path
from tqdm import tqdm
import tempfile

In [3]:

# 配置
OUTPUT_DIR = 'dataset'
TRAIN_PAIRS = 5000
VAL_PAIRS = 500
PATCH_SIZE = 50
TARGET_SIZE = 16
STRIDE = 10
TV_TOP_RATIO = 0.1


In [4]:

# 确保输出目录存在
os.makedirs(os.path.join(OUTPUT_DIR, 'train', 'blurred'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'train', 'target'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'val', 'blurred'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'val', 'target'), exist_ok=True)


In [5]:

def compute_tv(patch):
    gray = cv2.cvtColor(patch, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    return np.sum(np.abs(sobelx) + np.abs(sobely))


In [6]:

def defocus_kernel(radius):
    if radius == 0:
        return np.array([[1.0]])
    d = int(2*radius + 1)
    kernel = np.zeros((d, d))
    center = radius
    y, x = np.ogrid[:d, :d]
    mask = (x - center)**2 + (y - center)**2 <= radius**2
    kernel[mask] = 1.0
    kernel /= kernel.sum()
    return kernel


In [7]:

def motion_kernel(size):
    kernel = np.zeros((size, size))
    cx, cy = size//2, size//2
    x, y = cx, cy
    path = [(x, y)]
    for _ in range(size-1):
        dx, dy = random.choice([(1,0), (-1,0), (0,1), (0,-1), (1,1), (1,-1), (-1,1), (-1,-1)])
        x = max(0, min(size-1, x+dx))
        y = max(0, min(size-1, y+dy))
        path.append((x, y))
    for (x, y) in set(path):
        kernel[y, x] = 1.0
    kernel /= kernel.sum()
    return kernel


In [8]:

def process_and_save(clean_patch, file_id, subset):
    """
    clean_patch: 50x50 RGB 浮点 [0,1]
    file_id: 6位数字字符串
    subset: 'train' 或 'val'
    """
    # 1. 几何变换（旋转）
    angle = np.random.normal(0, 4)
    M = cv2.getRotationMatrix2D((PATCH_SIZE/2, PATCH_SIZE/2), angle, 1)
    transformed = cv2.warpAffine(clean_patch, M, (PATCH_SIZE, PATCH_SIZE), borderMode=cv2.BORDER_REPLICATE)
    
    # 2. 提取目标 (中心16x16)
    start = (PATCH_SIZE - TARGET_SIZE) // 2
    target = transformed[start:start+TARGET_SIZE, start:start+TARGET_SIZE].copy()
    
    # 3. 生成模糊核
    if random.random() < 0.5:
        r = random.uniform(0, 4)
        kernel = defocus_kernel(r)
    else:
        s = random.randint(5, 21)
        kernel = motion_kernel(s)
    
    blurred = cv2.filter2D(transformed, -1, kernel)
    
    # 4. 加噪声
    sigma = random.uniform(0, 7/255)
    noise = np.random.normal(0, sigma, blurred.shape)
    blurred = np.clip(blurred + noise, 0, 1)
    
    # 5. 量化模拟 (实际保存时转为uint8)
    blurred_uint8 = (blurred * 255).astype(np.uint8)
    target_uint8 = (target * 255).astype(np.uint8)
    
    # 6. 保存为PNG
    blurred_path = os.path.join(OUTPUT_DIR, subset, 'blurred', f'{file_id}.png')
    target_path = os.path.join(OUTPUT_DIR, subset, 'target', f'{file_id}.png')
    cv2.imwrite(blurred_path, cv2.cvtColor(blurred_uint8, cv2.COLOR_RGB2BGR))  # OpenCV 使用BGR
    cv2.imwrite(target_path, cv2.cvtColor(target_uint8, cv2.COLOR_RGB2BGR))
    
   
    return {'file_id': file_id, 'blur_type': 'defocus' if random.random()<0.5 else 'motion', 'sigma': sigma}


In [9]:
import glob

candidate_patches = []
original_folder = r"D:\CNN_for_Direct_Text_Deblurring\Code\dataset\original"
pdf_files = glob.glob(os.path.join(original_folder, "*.pdf"))
print(len(pdf_files))

42


In [10]:
with tempfile.TemporaryDirectory() as temp_dir:
    for pdf in tqdm(pdf_files):
        dpi = random.randint(210, 250)
        # 获取 PDF 总页数
        info = pdfinfo_from_path(pdf, userpw=None, poppler_path=None)  
        total_pages = info["Pages"]
        pdf_name = os.path.basename(pdf)
        print(f"Processing {pdf_name}...")
        # 逐页处理
        for page_num in range(1, total_pages + 1):
    
                paths = convert_from_path(
                    pdf, 
                    dpi=dpi, 
                    first_page=page_num, 
                    last_page=page_num,
                    paths_only=True,
                    poppler_path=None  # 如果有需要，指定 poppler 的 bin 路径
                )
                img_path = paths[0]      
                # 将 PIL Image 转换为 numpy 数组
                img_np = np.array(img_path)
                img_np = cv2.cvtColor(img_np, cv2.COLOR_BGR2RGB)
            # 下采样2倍
                h, w = img_np.shape[:2]
                img_np = cv2.resize(img_np, (w//2, h//2), interpolation=cv2.INTER_CUBIC)
                # 滑动窗口计算TV
                tv_list = []
                for y in range(0, img_np.shape[0]-PATCH_SIZE, STRIDE):
                    for x in range(0, img_np.shape[1]-PATCH_SIZE, STRIDE):
                        patch = img_np[y:y+PATCH_SIZE, x:x+PATCH_SIZE]
                        tv = compute_tv(patch)
                        tv_list.append((tv, y, x))
                # 取前10%
                tv_list.sort(reverse=True)
                top = tv_list[:max(1, int(len(tv_list)*TV_TOP_RATIO))]
                for _, y, x in top:
                    patch = img_np[y:y+PATCH_SIZE, x:x+PATCH_SIZE].astype(np.float32) / 255.0
                    candidate_patches.append(patch)
            
print(f"Collected {len(candidate_patches)} candidate patches.")


  0%|          | 0/42 [00:00<?, ?it/s]

Processing 06299.pdf...


  2%|▏         | 1/42 [00:09<06:19,  9.26s/it]

Processing 0910.4610v1.pdf...


  5%|▍         | 2/42 [00:10<03:11,  4.80s/it]

...

Processing Zhang_Dynamic_Scene_Deblurring_CVPR_2018_paper.pdf...


 98%|█████████▊| 41/42 [05:55<00:13, 13.41s/it]

Processing Zhang_Pixel_Screening_Based_Intermediate_Correction_for_Blind_Deblurring_CVPR_2022_paper.pdf...


100%|██████████| 42/42 [06:08<00:00,  8.78s/it]

Collected 521093 candidate patches.


In [11]:

# 随机打乱并分配训练/验证
random.shuffle(candidate_patches)
train_patches = candidate_patches[:TRAIN_PAIRS]
val_patches = candidate_patches[TRAIN_PAIRS:TRAIN_PAIRS+VAL_PAIRS]

# 处理训练集
metadata_train = []
for idx, patch in enumerate(tqdm(train_patches, desc='Train')):
    file_id = f"{idx+1:06d}"
    meta = process_and_save(patch, file_id, 'train')
    metadata_train.append(meta)

# 处理验证集
metadata_val = []
for idx, patch in enumerate(tqdm(val_patches, desc='Val')):
    file_id = f"{idx+1:06d}"
    meta = process_and_save(patch, file_id, 'val')
    metadata_val.append(meta)


Train:   0%|          | 0/5000 [00:00<?, ?it/s]C:\Users\Administrator\AppData\Local\Temp\ipykernel_11576\2802976263.py:10: RuntimeWarning: invalid value encountered in divide
  kernel /= kernel.sum()
C:\Users\Administrator\AppData\Local\Temp\ipykernel_11576\1311796566.py:32: RuntimeWarning: invalid value encountered in cast
  blurred_uint8 = (blurred * 255).astype(np.uint8)
Val: 100%|██████████| 500/500 [00:02<00:00, 245.61it/s]
